In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('./')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

In [2]:
batch_size = 128          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = False   # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
# num_epochs = 3             # number of training epochs
num_epochs = 25
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

vocab_size = len(data_loader.dataset.vocab)

encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

params = list(decoder.parameters()) + list(encoder.embed.parameters())

optimizer = torch.optim.Adam(params, lr=0.001)

total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

annotations file:  data/annotations/captions_train2014.json
loading annotations into memory...
Done (t=0.21s)
creating index...
index created!
[0/96057] Tokenizing captions...
loading annotations into memory...


  1%|          | 664/96057 [00:00<00:14, 6632.72it/s]

Done (t=0.21s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 96057/96057 [00:13<00:00, 7158.88it/s]
Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /tmp/xdg-cache/torch/checkpoints/resnet152-b121ed2d.pth
100%|██████████| 230M/230M [00:09<00:00, 25.3MB/s] 


In [3]:
vocab_size

4640

In [4]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

In [5]:
f = open(log_file, 'a')

encoder_file = 'encoder-751.pkl'
decoder_file = 'decoder-751.pkl'
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

for epoch in range(25, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
        if i_step % 200 == 0 or i_step == total_step:
            torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % i_step))
            torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % i_step))
            

f.close()

Epoch [25/25], Step [100/751], Loss: 1.0706, Perplexity: 2.9171
Epoch [25/25], Step [200/751], Loss: 1.1164, Perplexity: 3.0537
Epoch [25/25], Step [300/751], Loss: 1.1090, Perplexity: 3.0314
Epoch [25/25], Step [400/751], Loss: 1.0274, Perplexity: 2.7939
Epoch [25/25], Step [500/751], Loss: 1.0347, Perplexity: 2.814310
Epoch [25/25], Step [600/751], Loss: 1.0870, Perplexity: 2.9655
Epoch [25/25], Step [700/751], Loss: 1.4386, Perplexity: 4.2150
Epoch [25/25], Step [751/751], Loss: 1.0546, Perplexity: 2.8708